In [ ]:
import tensorflow as tf
import numpy as np
import math
import pandas as pd
import random
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import pyodbc
from sklearn.model_selection import train_test_split

In [ ]:
# Input weights initialization function
def xavier_init(fan_in, fan_out, constant = 1):
    low = -constant * np.sqrt(6.0 / (fan_in + fan_out))
    high = constant * np.sqrt(6.0 / (fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), minval = low, maxval = high, dtype = tf.float32)



In [ ]:
class AdditiveGaussianNoiseAutoencoder(object):

    # Input
    # n_input - dimension of input
    # n_hidden - number of hidden nodes
    # transfer_function - activation function to use
    # optimizer - Optimizer algorithm to use to train model
    # scale - training alpha?

    def __init__(self, n_input, n_hidden, transfer_function = tf.nn.sigmoid, optimizer = tf.train.AdamOptimizer(),
                 scale = 0.1, sparsity_percentage = 0.05, reg = 0.0):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32)
        self.training_scale = scale
        network_weights = self._initialize_weights()
        self.weights = network_weights
        self.sparsity_level= np.repeat([sparsity_percentage], self.n_hidden).astype(np.float32)
        self.sparse_reg = reg

        # model
        self.x = tf.placeholder(tf.float32, [None, self.n_input])
        
        
        # Need to wrap this function with competitive layer for KATE
        # after the transfer function
        self.hidden = self.transfer(
            tf.add(
                tf.matmul(
                    self.x + scale * tf.random_normal((n_input,)),
                    self.weights['w1']
                ),
                self.weights['b1']
            )
        )
        self.reconstruction = tf.add(
            tf.matmul(
                self.hidden, 
                self.weights['w2']
            ), 
            self.weights['b2']
        )

        # cost
        self.cost = 0.5 * tf.reduce_sum(
            tf.pow(
                tf.subtract(
                        self.reconstruction, self.x
                ), 
                2.0
            )
        )
            
        self.optimizer = optimizer.minimize(self.cost)

        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)

    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.Variable(xavier_init(self.n_input, self.n_hidden))
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden], dtype = tf.float32))
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden, self.n_input], dtype = tf.float32))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input], dtype = tf.float32))
        return all_weights

    def partial_fit(self, X):
        cost, opt = self.sess.run((self.cost, self.optimizer), feed_dict = {self.x: X,
                                                                            self.scale: self.training_scale
                                                                            })
        return cost

    def kl_divergence(self, p, p_hat):
        return tf.reduce_mean(
            p * tf.log(p) - p * tf.log(p_hat) + (1 - p) * tf.log(1 - p) - (1 - p) * tf.log(1 - p_hat))

    def calc_total_cost(self, X):
        return self.sess.run(self.cost, feed_dict = {self.x: X})

    def transform(self, X):
        return self.sess.run(self.hidden, feed_dict = {self.x: X,
                                                       self.scale: self.training_scale
                                                       })

    def kCompetitiveLayer(self, k, P, activations, alpha):

        ## TODO
        # Confirm dimensions and input types
        # Maintain activation index for mapping to original order


        # sort positive neurons from activation
        z_pos = tf.sort( tf.select( tf.where( x > 0, x = activations)), direction = 'DESCENDING')
        # sort negative neurons from activation
        z_neg = tf.sort( tf.select( tf.where( x < 0, x = activations)), direction = 'ASCENDING')
        
        P = tf.size(z_pos)
        N = tf.size(z_neg)

        # Distribute postive activations
        if P - np.floor(k/2) > 0:
            energy_pos = tf.math.reduce_sum( z_pos)
            for i in range( P - np.floor(k/2) + 1, P):
                z_pos[i] += alpha * energy_pos
            for i in range(1, P - np.floor(k/2)):
                z_pos[i] = 0

        # Distribute negative activations
        if N - np.floor(k/2) > 0:
            energy_neg = tf.math.reduce_sum( z_neg)
            for i in range( N - np.floor(k/2) + 1, N):
                z_pos[i] += alpha * energy_neg
            for i in range(1, N - np.floor(k/2)):
                z_pos[i] = 0

        return activations

    def generate(self, hidden = None):
        if hidden is None:
            hidden = np.random.normal(size = self.weights["b1"])
        return self.sess.run(self.reconstruction, feed_dict = {self.hidden: hidden})

    def reconstruct(self, X):
        return self.sess.run(self.reconstruction, feed_dict = {self.x: X,
                                                               self.scale: self.training_scale
                                                               })

    def getWeights(self):
        return self.sess.run(self.weights['w1'])

    def getBiases(self):
        return self.sess.run(self.weights['b1'])



In [ ]:
def get_random_block_from_data(data, batch_size):
    start_index = np.random.randint(0, len(data) - batch_size)
    return data[start_index:(start_index + batch_size)]


In [ ]:
# Get distince courses
query = "WITH unique_courses AS ( SELECT DISTINCT [Course ID] AS course_id FROM course_data_manuals ), log_counts AS ( SELECT [Course ID],[word],LOG(1 + COUNT(*)) as cases FROM [UCM].[dbo].[course_data_catalogues] L INNER JOIN unique_courses R ON L.[Course ID] = R.course_id WHERE L.word <> '' GROUP BY [Course ID],[word] ), max_cases AS ( 	SELECT MAX(cases) AS max_cases  FROM log_counts ) SELECT L.[Course ID] as course_id, L.word, L.cases / R.max_cases AS word_count FROM log_counts L, max_cases R "

conn = pyodbc.connect(
    'Driver={SQL Server};'
    'Server=DESKTOP-8LSE8HT;'
    'Database=UCM;'
    'Trusted_Connection=yes;'
)


df = pd.read_sql_query(query, conn)

df = df.pivot(index='course_id', columns = 'word' ,values='word_count').fillna(0)


X_train, X_test = train_test_split(df, test_size = 0.2)

In [ ]:
#mnist = input_data.read_data_sets('MNIST_data', one_hot = True)


#X_train = mnist.train.images
#X_test = mnist.test.images

n_samples = len(X_train)
training_epochs = 100
batch_size = len(X_test)
display_step = 1000

autoencoder = AdditiveGaussianNoiseAutoencoder(n_input = X_train[:1].size,
                                               n_hidden = 50,
                                               transfer_function = tf.nn.sigmoid,
                                               optimizer = tf.train.AdamOptimizer(learning_rate = 0.001),
                                               scale = 0.01,
                                               sparsity_percentage = 1.0)


In [ ]:

autoencoder_costs = {}
for sparsity_num in [0.05, 0.1, 0.25, 0.5, 1.0]:
    all_costs = []
    for hidden_num in range(1,100,5):
        autoencoder = AdditiveGaussianNoiseAutoencoder(n_input = X_train[:1].size,
                                                    n_hidden = hidden_num,
                                                    transfer_function = tf.nn.sigmoid,
                                                    optimizer = tf.train.AdamOptimizer(learning_rate = 0.001),
                                                    scale = 0.01,
                                                    sparsity_percentage = sparsity_num)

        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(n_samples / batch_size)
            # Loop over all batches
            for i in range(total_batch):
                batch_xs = get_random_block_from_data(X_train, batch_size)

                # Fit training using batch data
                cost = autoencoder.partial_fit(batch_xs)
                # Compute average loss
                avg_cost += cost / n_samples * batch_size

            # Display logs per epoch step
            #if epoch % display_step == 0:
            #    print("Epoch:", '%04d' % (epoch + 1), "cost=", avg_cost)
        all_costs.append(autoencoder.calc_total_cost(X_test))
        print("Total cost (Hidden Units = " + str(hidden_num) + "): " + str(autoencoder.calc_total_cost(X_test)))

    autoencoder_costs[sparsity_num] = all_costs


In [ ]:

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = go.Figure()

for key, vals in autoencoder_costs.items():
    fig.add_trace(
        go.Scatter(
            y = vals,
            x = list(range(len(vals))),
            name = "Sparsity = " + str(key)
        )
    )

fig.update_layout(
    title = "Autoencoding Text data",
    xaxis_title="Number of hidden units",
    yaxis_title="Cost"
)
fig.update_yaxes(range=[70,85])

fig.show()

In [ ]:
# Plotting methods

# input weights
wts = autoencoder.getWeights()
dim = math.ceil(math.sqrt(autoencoder.n_hidden))
plt.figure(1, figsize=(dim, dim))
for i in range(0,autoencoder.n_hidden):
    im = wts.flatten()[i::autoencoder.n_hidden].reshape((28,28))
    plt.subplot(dim, dim, i+1)
    #plt.title('Feature Weights ' + str(i))
    plt.imshow(im, cmap="gray", clim=(-1.0, 1.0))
plt.show()

predicted_imgs = autoencoder.reconstruct(X_test[:100])

# plot the reconstructed images
plt.figure(1, figsize=(10, 10))
plt.title('Autoencoded Images')
for i in range(0,100):
    im = predicted_imgs[i].reshape((28,28))
    plt.subplot(10, 10, i+1)
    plt.imshow(im, cmap="gray", clim=(0.0, 1.0))
plt.show()

In [ ]:
class Autoencoder(object):

    # Input
    # n_input - dimension of input
    # n_hidden - number of hidden nodes
    # transfer_function - activation function to use
    # optimizer - Optimizer algorithm to use to train model
    # scale - training alpha?

    def __init__(self, n_input, n_hidden, transfer_function = tf.nn.sigmoid, optimizer = tf.train.AdamOptimizer(), reg = 0.0):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        network_weights = self._initialize_weights()
        self.weights = network_weights
        self.sparse_reg = reg

        # model
        self.x = tf.placeholder(tf.float32, [None, self.n_input])
                
        # Need to wrap this function with competitive layer for KATE
        # after the transfer function
        self.hidden = self.transfer(
            tf.add(
                tf.matmul(
                    self.x,
                    self.weights['w1']
                ),
                self.weights['b1']
            )
        )
        self.reconstruction = tf.add(
            tf.matmul(
                self.hidden, 
                self.weights['w2']
            ), 
            self.weights['b2']
        )

        # cost, squared sum
        self.cost = 0.5 * tf.reduce_sum(
            tf.pow(
                tf.subtract(
                        self.reconstruction, 
                        self.x
                ), 
                2.0
            )
        )
            
        self.optimizer = optimizer.minimize(self.cost)

        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)

    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.Variable(xavier_init(self.n_input, self.n_hidden))
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden], dtype = tf.float32))
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden, self.n_input], dtype = tf.float32))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input], dtype = tf.float32))
        return all_weights

    def partial_fit(self, X):
        cost, opt = self.sess.run((self.cost, self.optimizer), feed_dict = {self.x: X})
        return cost

    def calc_total_cost(self, X):
        return self.sess.run(self.cost, feed_dict = {self.x: X})

    def transform(self, X):
        return self.sess.run(self.hidden, feed_dict = {self.x: X})

    def generate(self, hidden = None):
        if hidden is None:
            hidden = np.random.normal(size = self.weights["b1"])
        return self.sess.run(self.reconstruction, feed_dict = {self.hidden: hidden})

    def reconstruct(self, X):
        return self.sess.run(self.reconstruction, feed_dict = {self.x: X})

    def getWeights(self):
        return self.sess.run(self.weights['w1'])

    def getBiases(self):
        return self.sess.run(self.weights['b1'])


In [ ]:
class kDenseAutoencoder(object):

    # Input
    # n_input - dimension of input
    # n_hidden - number of hidden nodes
    # transfer_function - activation function to use
    # optimizer - Optimizer algorithm to use to train model
    # scale - training alpha?

    def __init__(self, n_input, n_hidden, transfer_function = tf.nn.sigmoid, optimizer = tf.train.AdamOptimizer(), reg = 0.0, k = 10):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        network_weights = self._initialize_weights()
        self.weights = network_weights
        self.sparse_reg = reg
        self.k = k

        # model
        self.x = tf.placeholder(tf.float32, [None, self.n_input])
                
        # Need to wrap this function with competitive layer for KATE
        # after the transfer function
        self.hidden = self.transfer(
            tf.add(
                tf.matmul(
                    self.x,
                    self.weights['w1']
                ),
                self.weights['b1']
            )
        )
        self.values, self.indices = tf.math.top_k(self.hidden, k = self.k)

        print(self.values.eval(), self.indices.eval())

        self.kDenseHidden = tf.SparseTensor( self.indices, self.values, tf.shape(self.n_hidden, out_type = tf.float64))
        self.reconstruction = tf.add(
            tf.matmul(
                self.kDenseHidden, 
                self.weights['w2']
            ), 
            self.weights['b2']
        )

        # cost, squared sum
        self.cost = 0.5 * tf.reduce_sum(
            tf.pow(
                tf.subtract(
                        self.reconstruction, 
                        self.x
                ), 
                2.0
            )
        )
            
        self.optimizer = optimizer.minimize(self.cost)

        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)

    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.Variable(xavier_init(self.n_input, self.n_hidden))
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden], dtype = tf.float32))
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden, self.n_input], dtype = tf.float32))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input], dtype = tf.float32))
        return all_weights

    def partial_fit(self, X):
        cost, opt = self.sess.run((self.cost, self.optimizer), feed_dict = {self.x: X})
        return cost

    def calc_total_cost(self, X):
        return self.sess.run(self.cost, feed_dict = {self.x: X})

    def transform(self, X):
        return self.sess.run(self.hidden, feed_dict = {self.x: X})

    def generate(self, hidden = None):
        if hidden is None:
            hidden = np.random.normal(size = self.weights["b1"])
        return self.sess.run(self.reconstruction, feed_dict = {self.hidden: hidden})

    def reconstruct(self, X):
        return self.sess.run(self.reconstruction, feed_dict = {self.x: X})

    def getWeights(self):
        return self.sess.run(self.weights['w1'])

    def getBiases(self):
        return self.sess.run(self.weights['b1'])


In [ ]:
n_samples = len(X_train)
training_epochs = 100
batch_size = len(X_test)
display_step = 1000


basic_autoencoder = {}
all_costs = []
sparsity_num = 0.05
hidden_num = 100
autoencoder = kDenseAutoencoder(
    n_input = X_train[:1].size,
    n_hidden = hidden_num,
    transfer_function = tf.nn.sigmoid,
    optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
)


for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(n_samples / batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_xs = get_random_block_from_data(X_train, batch_size)

        # Fit training using batch data
        cost = autoencoder.partial_fit(batch_xs)
        # Compute average loss
        avg_cost += cost / n_samples * batch_size

    # Display logs per epoch step
    #if epoch % display_step == 0:
    #    print("Epoch:", '%04d' % (epoch + 1), "cost=", avg_cost)
all_costs.append(autoencoder.calc_total_cost(X_test))
print("Total cost (Hidden Units = " + str(hidden_num) + "): " + str(autoencoder.calc_total_cost(X_test)))


# 74.562
